In [ ]:
!pip3 install autoawq

In [ ]:
!cd /workspace/repos && git clone https://github.com/arcee-ai/mergekit.git && cd mergekit && pip install -e .

In [ ]:
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM

model_a_name = "Qwen/QwQ-32B"
model_b_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

model_a = AutoModelForCausalLM.from_pretrained(model_a_name)
tokenizer_a = AutoTokenizer.from_pretrained(model_a_name)
del model_a, tokenizer_a
gc.collect()

model_b = AutoModelForCausalLM.from_pretrained(model_b_name)
tokenizer_b = AutoTokenizer.from_pretrained(model_b_name)
del model_b, tokenizer_b
gc.collect()

In [ ]:
!mergekit-yaml /workspace/repos/agentic-ai/tutorials/merge_models.yml /workspace/model_merged_daretie/

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = "/workspace/model_merged_daretie/"
quant_path = "/workspace/model_merged_daretie_quant/"


model = AutoAWQForCausalLM.from_pretrained(
    model_path, **{"low_cpu_mem_usage": True, "use_cache": False}
)

quant_config={
  "zero_point": True,
  "q_group_size": 128,
  "w_bit": 4,
  "version": "GEMM"
}

# Quantize
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')